<a href="https://colab.research.google.com/github/gangulasreeja/ISA/blob/main/Copy_of_Legal_Pegasus_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import nltk

In [ ]:
!pip install nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
max_input = 512
max_target = 128
batch_size = 3
model_checkpoints = "google/pegasus-large"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
from datasets import load_dataset

In [ ]:
ds = load_dataset("ninadn/indian-legal")

train.csv:   0%|          | 0.00/210M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/3.26M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
def preprocess_data(data_to_process):
    inputs = data_to_process['Text']
    targets = data_to_process['Summary']
    inputs = [" " if text is None else text for text in inputs]
    targets = [" " if text is None else text for text in targets]
    model_inputs = tokenizer(inputs, max_length=max_input, padding='max_length', truncation=True)
    targets_tokenized = tokenizer(text_target=targets, max_length=max_target, padding='max_length', truncation=True)
    model_inputs['labels'] = targets_tokenized['input_ids']
    return model_inputs
tokenize_data = ds.map(preprocess_data, batched=True)

Map:   0%|          | 0/7030 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
num_valid = 200
train_valid_split = tokenize_data['train'].train_test_split(test_size=num_valid, shuffle=True, seed=42)

# Rename keys for clarity
tokenize_data['train'] = train_valid_split['train']
tokenize_data['valid'] = train_valid_split['test']

# Print sizes to confirm
print(f"Train dataset size: {len(tokenize_data['train'])}")
print(f"Validation dataset size: {len(tokenize_data['valid'])}")
print(f"Text dataset size: {len(tokenize_data['test'])}")

Train dataset size: 6830
Validation dataset size: 200
Text dataset size: 100


In [ ]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [ ]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)
batch_size = 3

In [ ]:
!pip install evaluate
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score
metric = evaluate.load("rouge")

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=a59ea5451c71055d75148ed9210510d8af6d9f6a363fa68d3455267c894eced3
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [ ]:
args = transformers.Seq2SeqTrainingArguments(
    'legal-summ-pegasus-large',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size= 1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=6,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True,
    # Disable gradient clipping
    max_grad_norm=None
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = transformers.Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['valid'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

<ipython-input-17-05b190274e21>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Seq2SeqTrainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nandiny6504 (nandiny6504-nitw) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.122200,2.082177,43.830100,22.286800,30.014900,30.081800,220.770000
2,1.962300,2.026925,46.657100,23.492000,31.621300,31.705700,171.915000
3,1.917500,2.003260,47.894700,23.714400,32.092100,32.165700,148.910000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


In [ ]:
pip install transformers datasets huggingface_hub

In [ ]:
from huggingface_hub import login

login()

In [ ]:
model.save_pretrained('./legal-summ-pegasus-large')
tokenizer.save_pretrained('./legal-summ-pegasus-large')

In [ ]:
from huggingface_hub import HfApi, HfFolder

# Push the model and tokenizer to your Hugging Face account
api = HfApi()
api.upload_folder(
    folder_path="./legal-summ-pegasus",
    path_in_repo=".",
    repo_id="SaiNandiniRangaa/my_model_repo",
    repo_type="model"
)